# <div align="center" style="color: #ff5733;">New Pos Addition and Mapping</div>

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\SIL Sales\Shin Nebria\POS_Addition\Data\pos_add_20240725.csv")
df.shape

(7, 8)

In [3]:
# df = pd.read_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\SIL Sales\Shin Nebria\POS_Addition\Data\POS_mapping_data May042024.xlsx", sheet_name = 'Sheet1')
# df.head(10)

In [4]:
df.head()

,POS NAME,STORE LOCATION,STATUS,DATE ACTIVATION,PURPLE KEY,REGION,TSM,Brand
0,Rulls Waltermart Bicutan,Standalone,For Opening,"July 26, 2024",MX1EHIO,NCR South 2,Grace Belarmino,Rulls
1,Techlife Robinsons Galleria South,Robinsons Galleria South,For Opening,"July 26, 2024",MY295QO,South Luzon 1 (A),Aubrey Lalong Isip,Rulls
2,Tecno Robinsons Galleria South,Robinsons Galleria South,For Opening,"July 26, 2024",MVNGWHA,South Luzon 1 (A),Aubrey Lalong Isip,Rulls
3,Stormtech Gaisano Mall Tabunok,Gaisano Fiesta Mall,POS Operational,"July 26, 2024",MLK8KS4,South Cebu,Alexander Limosnero,Rulls
4,Audionet Store Gaisano Mall Carcar,Grand Mall Carcar,For Opening,"July 26, 2024",MIXB34S,South Cebu,Alexander Limosnero,Rulls


In [5]:
df.rename(columns = {'Brand': 'BrandStores'}, inplace = True)
df.columns

Index(['POS NAME', 'STORE LOCATION', 'STATUS', 'DATE ACTIVATION', 'PURPLE KEY',
       'REGION', 'TSM', 'BrandStores'],
      dtype='object')

In [6]:
df['BrandStores'].value_counts()

BrandStores
Rulls    7
Name: count, dtype: int64

In [7]:
df['PURPLE KEY'].value_counts()

PURPLE KEY
MX1EHIO    1
MY295QO    1
MVNGWHA    1
MLK8KS4    1
MIXB34S    1
M1YFI29    1
M4RWEBV    1
Name: count, dtype: int64

In [8]:
df = df[['BrandStores', 'POS NAME', 'STORE LOCATION', 'STATUS', 'DATE ACTIVATION', 'PURPLE KEY', 'REGION', 'TSM']]
df

,BrandStores,POS NAME,STORE LOCATION,STATUS,DATE ACTIVATION,PURPLE KEY,REGION,TSM
0,Rulls,Rulls Waltermart Bicutan,Standalone,For Opening,"July 26, 2024",MX1EHIO,NCR South 2,Grace Belarmino
1,Rulls,Techlife Robinsons Galleria South,Robinsons Galleria South,For Opening,"July 26, 2024",MY295QO,South Luzon 1 (A),Aubrey Lalong Isip
2,Rulls,Tecno Robinsons Galleria South,Robinsons Galleria South,For Opening,"July 26, 2024",MVNGWHA,South Luzon 1 (A),Aubrey Lalong Isip
3,Rulls,Stormtech Gaisano Mall Tabunok,Gaisano Fiesta Mall,POS Operational,"July 26, 2024",MLK8KS4,South Cebu,Alexander Limosnero
4,Rulls,Audionet Store Gaisano Mall Carcar,Grand Mall Carcar,For Opening,"July 26, 2024",MIXB34S,South Cebu,Alexander Limosnero
5,Rulls,Rulls Exhibit Robinsons Galleria Cebu,Robinsons Galleria Cebu,For Opening,"July 26, 2024",M1YFI29,South Cebu,Alexander Limosnero
6,Rulls,Vivo Kiosk Grandmall Liloan,Gaisano Grand Mall Liloan,For Opening,"July 26, 2024",M4RWEBV,North Cebu,Rona Jay Sinangote


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   BrandStores      7 non-null      object
 1   POS NAME         7 non-null      object
 2   STORE LOCATION   7 non-null      object
 3   STATUS           7 non-null      object
 4   DATE ACTIVATION  7 non-null      object
 5   PURPLE KEY       7 non-null      object
 6   REGION           7 non-null      object
 7   TSM              7 non-null      object
dtypes: object(8)
memory usage: 576.0+ bytes


In [10]:
df.rename(columns={'POS NAME':'POS', 'DATE ACTIVATION':'DateOpened', 'STORE LOCATION': 'StoreLocation', 'STATUS':'Status', 'PURPLE KEY':'PurpleKey', 'REGION':'Region'}, inplace = True)

In [11]:
df.head()

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
0,Rulls,Rulls Waltermart Bicutan,Standalone,For Opening,"July 26, 2024",MX1EHIO,NCR South 2,Grace Belarmino
1,Rulls,Techlife Robinsons Galleria South,Robinsons Galleria South,For Opening,"July 26, 2024",MY295QO,South Luzon 1 (A),Aubrey Lalong Isip
2,Rulls,Tecno Robinsons Galleria South,Robinsons Galleria South,For Opening,"July 26, 2024",MVNGWHA,South Luzon 1 (A),Aubrey Lalong Isip
3,Rulls,Stormtech Gaisano Mall Tabunok,Gaisano Fiesta Mall,POS Operational,"July 26, 2024",MLK8KS4,South Cebu,Alexander Limosnero
4,Rulls,Audionet Store Gaisano Mall Carcar,Grand Mall Carcar,For Opening,"July 26, 2024",MIXB34S,South Cebu,Alexander Limosnero


In [12]:
from datetime import datetime
df['DateOpened'] = pd.to_datetime(df['DateOpened'], format='%d-%m-%Y')
# Format the datetime object to YYYY-MM-DD
df['DateOpened'] = df['DateOpened'].dt.strftime('%Y-%m-%d')

ValueError: time data "July 26, 2024" doesn't match format "%d-%m-%Y", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [13]:
import pandas as pd

df['DateOpened'] = pd.to_datetime(df['DateOpened'], format='%B %d, %Y')


In [14]:
print(df['DateOpened'].dtype)
# datetime64[ns]
print(df['DateOpened'].iloc[0]) 
# 2024-01-15 00:00:00
df['DateOpened'] = df['DateOpened'].dt.strftime('%Y-%m-%d')
df.info()

datetime64[ns]
2024-07-26 00:00:00
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   BrandStores    7 non-null      object
 1   POS            7 non-null      object
 2   StoreLocation  7 non-null      object
 3   Status         7 non-null      object
 4   DateOpened     7 non-null      object
 5   PurpleKey      7 non-null      object
 6   Region         7 non-null      object
 7   TSM            7 non-null      object
dtypes: object(8)
memory usage: 576.0+ bytes


In [15]:
import pandas as pd

df['DateOpened'] = pd.to_datetime(df['DateOpened'], format='%d-%b-%y')

ValueError: time data "2024-07-26" doesn't match format "%d-%b-%y", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [16]:
print(df['DateOpened'].dtype)
# datetime64[ns]

object


In [17]:
print(df['DateOpened'].iloc[0]) 
# 2024-01-15 00:00:00

2024-07-26


In [18]:
df['DateOpened'] = df['DateOpened'].dt.strftime('%Y-%m-%d')
df.info()

AttributeError: Can only use .dt accessor with datetimelike values

In [19]:
df['DateOpened'] = pd.to_datetime(df['DateOpened'], format='%Y-%m-%d')

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   BrandStores    7 non-null      object        
 1   POS            7 non-null      object        
 2   StoreLocation  7 non-null      object        
 3   Status         7 non-null      object        
 4   DateOpened     7 non-null      datetime64[ns]
 5   PurpleKey      7 non-null      object        
 6   Region         7 non-null      object        
 7   TSM            7 non-null      object        
dtypes: datetime64[ns](1), object(7)
memory usage: 576.0+ bytes


In [21]:
df.head()

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
0,Rulls,Rulls Waltermart Bicutan,Standalone,For Opening,2024-07-26,MX1EHIO,NCR South 2,Grace Belarmino
1,Rulls,Techlife Robinsons Galleria South,Robinsons Galleria South,For Opening,2024-07-26,MY295QO,South Luzon 1 (A),Aubrey Lalong Isip
2,Rulls,Tecno Robinsons Galleria South,Robinsons Galleria South,For Opening,2024-07-26,MVNGWHA,South Luzon 1 (A),Aubrey Lalong Isip
3,Rulls,Stormtech Gaisano Mall Tabunok,Gaisano Fiesta Mall,POS Operational,2024-07-26,MLK8KS4,South Cebu,Alexander Limosnero
4,Rulls,Audionet Store Gaisano Mall Carcar,Grand Mall Carcar,For Opening,2024-07-26,MIXB34S,South Cebu,Alexander Limosnero


In [22]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1;"""
client.query(sq)
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1;"""
try:
    d = client.query(sq).to_dataframe(progress_bar_type='tqdm')
    print("Table still exists")
except:
    print("Table dropped")

Query is running:   0%|          |Table dropped


In [25]:
sq = """create table prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1
as 
select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING;
"""
client.query(sq)



QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=74a59546-294d-4990-9d03-450744bdc7c1>

In [26]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1;"""
d = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of d is:\t{d.shape}")

Job ID 5c3b0f1a-6387-423e-81c7-aa17dc7864bb successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of d is:	(845, 8)


In [27]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1"""

dfsil = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID cb5e9e97-b171-44de-a367-037754733a53 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


In [28]:
dfsil.BrandStores.value_counts().sort_index()

BrandStores
A2RJ                            3
Affordables                     2
Ansons                         15
ArkHome                         4
Automatic Centre               11
Chrisrence Home Appliances      2
Dizon Furniture                 1
ESLA                           51
FC Home                       109
Faith Glorious                  9
Gadgetmate                     10
Hardware Sugar                  2
Hazrat                          6
Home Along                     95
Home Factory                    2
Homeworks                      15
JSL                             7
JTG                             1
Jhunpyo                         8
Junliz                          8
KUM Home Center                 3
Kservico                       10
MemoXpress                     54
PHPS                            2
Power Mac Center                5
Presnet                        21
Prince and Aviel                3
RCB Mobile                      3
Regs Fone                       2
Ro

In [29]:
dfsil.shape

(845, 8)

In [30]:
# Set table_id to the ID of the table to append to.
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1"

job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND"
)

job = client.load_table_from_dataframe(
    df, table_id, job_config=job_config
)  

job.result()  # Wait for the job to complete.

table = client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

Loaded 852 rows and 8 columns to prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1


In [31]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1"""
df = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')
print(f"The shape of the dataframe is: {df.shape}")

Job ID 351b6ea6-98be-4e6a-a16e-f99468b0fc02 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is: (852, 8)


In [32]:
df.tail(10)

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
842,FC Home,FC Home Center Binan,Central Mall Laguna,POS Operational,2023-09-14,MYRK3EJ,South Luzon 2,Christian Klide Alberto
843,FC Home,FC Home Center San Pablo,Standalone,POS Operational,2023-09-14,MADP0JZ,South Luzon 2,Christian Klide Alberto
844,FC Home,FC Home Center Sta Cruz,Standalone,POS Operational,2023-09-14,M1ZYHKU,South Luzon 2,Christian Klide Alberto
845,FC Home,FC Home Center Cabuyao 1,Standalone,POS Operational,2023-09-14,M5D8JOH,South Luzon 2,Christian Klide Alberto
846,FC Home,FC Home Center Mamatid,Standalone,POS Operational,2023-09-14,MC31AV7,South Luzon 2,Christian Klide Alberto
847,FC Home,FC Home Center Siniloan,Standalone,POS Operational,2023-09-15,MANB6XQ,South Luzon 2,Christian Klide Alberto
848,Workstation,Workstation Home Appliances Pavilion Mall 1,Pavillon Mall,POS Operational,2023-10-13,MN8MWPJ,South Luzon 2,Christian Klide Alberto
849,Workstation,Workstation Home Appliances Target Mall,Target Mall,POS Operational,2023-10-13,M8I4QGT,South Luzon 2,Christian Klide Alberto
850,Workstation,Workstation Home Appliances Central Mall,Central Mall,POS Operational,2023-10-13,MTNCOR5,South Luzon 2,Christian Klide Alberto
851,Xtreme,Xtreme Appliances Calamba,Carmel Mall,POS Operational,2023-10-15,MUBPW04,South Luzon 2,Christian Klide Alberto


In [33]:
df['BrandStores'].value_counts().sort_index()

BrandStores
A2RJ                            3
Affordables                     2
Ansons                         15
ArkHome                         4
Automatic Centre               11
Chrisrence Home Appliances      2
Dizon Furniture                 1
ESLA                           51
FC Home                       109
Faith Glorious                  9
Gadgetmate                     10
Hardware Sugar                  2
Hazrat                          6
Home Along                     95
Home Factory                    2
Homeworks                      15
JSL                             7
JTG                             1
Jhunpyo                         8
Junliz                          8
KUM Home Center                 3
Kservico                       10
MemoXpress                     54
PHPS                            2
Power Mac Center                5
Presnet                        21
Prince and Aviel                3
RCB Mobile                      3
Regs Fone                       2
Ro

In [34]:
# Find duplicate values in 'POS' column
duplicates = df[df.duplicated('POS', keep=False)]

# Print the entire rows for duplicate entries in 'POS'
if not duplicates.empty:
    print("Duplicate values in 'POS' column:")
    duplicates.to_csv("Temp.csv")
else:
    print("No duplicate values found in 'POS' column.")
    
dd = pd.read_csv("Temp.csv")
dd.sort_values(by='POS')

Duplicate values in 'POS' column:


,Unnamed: 0,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
0,8,Hardware Sugar,Hardware Sugar Makati,NaN,NaN,2022-07-04,M9ZK6IU,NaN,NaN
1,9,Hardware Sugar,Hardware Sugar Makati,NaN,NaN,2022-07-04,MV4UO0X,NaN,NaN
15,815,Home Along,Home Along - Mayon,Standalone,POS Operational,2022-07-01,MN3GL66,NCR North 2,Ben Antonio Lacsamana
14,812,Home Along,Home Along - Mayon,Standalone,POS Operational,2022-07-01,MGP2AQI,NCR North,Ben Antonio Lacsamana
16,821,Home Along,Home Along Malanday,NaN,NaN,2022-07-11,MURFAIV,NCR North,Ben Antonio Lacsamana
17,822,Home Along,Home Along Malanday,Standalone,POS Operational,2022-07-11,MNBW4RZ,NCR North,Ben Antonio Lacsamana
6,734,Home Along,Home Along Muntinlupa,Standalone,POS Operational,2022-03-26,MDGYHQJ,NCR South,Klein Justine Tamayo
7,735,Home Along,Home Along Muntinlupa,Standalone,POS Operational,2022-03-26,MU82X9Z,NCR South,Klein Justine Tamayo
9,749,Home Along,Home Along Puregold Taguig,Standalone,POS Operational,2022-07-01,MRGP0WQ,NCR South,Klein Justine Tamayo
10,750,Home Along,Home Along Puregold Taguig,Standalone,POS Operational,2022-07-01,MDO3KI4,NCR South,Klein Justine Tamayo


In [35]:
pd.set_option('display.max_rows', None)
# Count the occurrences of each value in the 'PurpleKey' column
purple_key_counts = df['PurpleKey'].value_counts()

# Filter to include only values with count >= 2
filtered_counts = purple_key_counts[purple_key_counts >= 2]

# Display the filtered counts
print(filtered_counts)


Series([], Name: count, dtype: int64)


In [36]:
df[df['PurpleKey']== 'MX1EHIO']

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
0,Rulls,Rulls Waltermart Bicutan,Standalone,For Opening,2024-07-26,MX1EHIO,NCR South 2,Grace Belarmino


In [37]:
sq = """drop table prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING;"""

client.query(sq)


QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=a51bb4cf-276a-4337-8551-f531c1d3975e>

In [38]:
sq = """create table prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING as select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1;"""
client.query(sq)


QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=6d4a7467-a3b0-4dc4-b7b0-7bb3da719475>

In [39]:
client.query("commit")

QueryJob<project=prj-prod-dataplatform, location=US, id=9f80a450-b681-43fb-a65a-a51a8058431c>

In [40]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING;"""

df = client.query(sq).to_dataframe(progress_bar_type='tqdm')
df.shape

Job ID c5f4da0a-98ba-46e9-af2d-36d4ef1ee853 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


(852, 8)

In [41]:
df[df['BrandStores']=='Rulls'].sort_values(by='DateOpened')

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
7,Rulls,Mobile Craze Town Center Mall Tanay,Craze Town Center Mall Tanay,For Opening,2024-04-15,M3ED6SF,NCR East,Albee David
485,Rulls,Rulls Ayala Mall Serin,Ayala Mall Serin,For Opening,2024-04-15,MQVP6H8,South Luzon 1 (A),Aubrey Lalong Isip
412,Rulls,Realme Apalit,Savemore Apalit,For Opening,2024-04-15,MXLK6VY,North Luzon 2,Rachelle Belmonte
408,Rulls,Rulls Apalit,Savemore Apalit,For Opening,2024-04-15,MA0C1DZ,North Luzon 2,Rachelle Belmonte
407,Rulls,Vivo Exhibit Marquee Mall,Marque Mall,For Opening,2024-04-15,MPORV4O,North Luzon 2,Rachelle Belmonte
406,Rulls,Vivo Apalit,Savemore Apalit,For Opening,2024-04-15,MQULK65,North Luzon 2,Rachelle Belmonte
404,Rulls,Rulls Marquee Mall,Marque Mall,For Opening,2024-04-15,MQKAO6D,North Luzon 2,Rachelle Belmonte
403,Rulls,Mobilecraze Marquee Mall,Marque Mall,For Opening,2024-04-15,MP1VR8S,North Luzon 2,Rachelle Belmonte
402,Rulls,Oppo Marquee Mall,Marque Mall,For Opening,2024-04-15,MYIF18A,North Luzon 2,Rachelle Belmonte
401,Rulls,Audionet Marquee Mall,Marque Mall,For Opening,2024-04-15,MCAZ46R,North Luzon 2,Rachelle Belmonte


In [42]:
df.head(10)

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
0,Xtreme,Xtreme Appliances Anabu,None,None,2022-08-27,MR2YDE8,None,None
1,Hardware Sugar,Hardware Sugar Makati,None,None,2022-07-04,MV4UO0X,None,None
2,Hardware Sugar,Hardware Sugar Makati,None,None,2022-07-04,M9ZK6IU,None,None
3,Home Along,Home Along Sumulong,None,None,2022-07-12,M2G78K4,None,None
4,Dizon Furniture,Dizon Furniture Tiendesitas,None,None,2022-07-23,MOE3JQH,None,None
5,Xtreme,Xtreme Appliances Lucky Gold Plaza,Lucky Gold Plaza,For Opening,2024-04-14,MO3AHGW,NCR East,Albee David
6,Ansons,Ansons Cainta,Q-Plaza Cainta,For Opening,2024-04-15,MDTBKRI,NCR East,Albee David
7,Rulls,Mobile Craze Town Center Mall Tanay,Craze Town Center Mall Tanay,For Opening,2024-04-15,M3ED6SF,NCR East,Albee David
8,Ansons,Ansons Ortigas,Standalone,For Opening,2024-04-15,MAQ1TK4,NCR East,Albee David
9,Rulls,Mobilecraze K2 PG Marikina,Puregold Marikina,For Opening,2024-04-15,MUFDX4A,NCR East,Albee David


In [43]:
df.sort_values('DateOpened', ascending=False)

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
568,Rulls,Audionet Parkmall,Parkmall Mandaue,For Opening,2024-08-12,MWE2YZF,North Cebu,Rona Jay Sinangote
645,Rulls,Rullst Inline Minglanilla,Gaisano Mall Minglanilla,For Opening,2024-08-12,M3Q0LYO,South Cebu,Alexander Limosnero
646,Rulls,Audionet Minglanilla,Gaisano Mall Minglanilla,For Opening,2024-08-12,M7P8T4Z,South Cebu,Alexander Limosnero
647,Rulls,Oppo Kiosk Minglanilla,Gaisano Mall Minglanilla,For Opening,2024-08-12,MWVUOSI,South Cebu,Alexander Limosnero
648,Rulls,Realme Kiosk Minglanilla,Gaisano Mall Minglanilla,For Opening,2024-08-12,M932V8C,South Cebu,Alexander Limosnero
649,Rulls,Vivo Kiosk Minglanilla,Gaisano Mall Minglanilla,For Opening,2024-08-12,ML0RC1E,South Cebu,Alexander Limosnero
650,Rulls,Rulls Minglanilla Exhibit,Gaisano Mall Minglanilla,For Opening,2024-08-12,MZL10XY,South Cebu,Alexander Limosnero
567,Rulls,Rulls Colonade Mandaue,Standalone,For Opening,2024-08-05,MLAFUKG,North Cebu,Rona Jay Sinangote
566,Rulls,Techsynergy Gmall Mandaue North,Gaisano Mall Mandaue,For Opening,2024-08-05,M9YQ1HP,North Cebu,Rona Jay Sinangote
565,Rulls,Rulls Liloan,Gaisano Mall Liloan,For Opening,2024-08-05,MFJ7H04,North Cebu,Rona Jay Sinangote


In [44]:
df[df['PurpleKey'] =='MX1EHIO']

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
164,Rulls,Rulls Waltermart Bicutan,Standalone,For Opening,2024-07-26,MX1EHIO,NCR South 2,Grace Belarmino


In [45]:
df.groupby(['BrandStores', 'DateOpened'])['PurpleKey'].nunique()

BrandStores                 DateOpened
A2RJ                        2024-02-05      3
Affordables                 2024-01-22      1
                            2024-04-12      1
Ansons                      2024-04-15     15
ArkHome                     2024-06-07      4
Automatic Centre            2023-07-15      6
                            2023-07-16      1
                            2023-07-22      4
Chrisrence Home Appliances  2024-07-13      2
Dizon Furniture             2022-07-23      1
ESLA                        2024-02-15     30
                            2024-04-22     21
FC Home                     2023-08-01     48
                            2023-09-01     20
                            2023-09-05      1
                            2023-09-14     25
                            2023-09-15     11
                            2023-11-10      1
                            2024-01-15      1
                            2024-03-26      1
                            2024-04-24   

In [46]:
df[df['BrandStores']=='Rulls']

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
7,Rulls,Mobile Craze Town Center Mall Tanay,Craze Town Center Mall Tanay,For Opening,2024-04-15,M3ED6SF,NCR East,Albee David
9,Rulls,Mobilecraze K2 PG Marikina,Puregold Marikina,For Opening,2024-04-15,MUFDX4A,NCR East,Albee David
10,Rulls,MBC Montalban Town Center,Montalban Town Center,For Opening,2024-04-15,MI9KLNA,NCR East,Albee David
11,Rulls,Fun Mobile Ayala Marikina,Ayala Malls Marikina,For Opening,2024-04-15,MK4CZP2,NCR East,Albee David
12,Rulls,Rulls Antipolo,Robinsons Antipolo,For Opening,2024-04-15,M8U05KZ,NCR East,Albee David
13,Rulls,Audionet Robinsons Metro East,Robinsons Metro East,For Opening,2024-04-15,M8BPO3Y,NCR East,Albee David
15,Rulls,Realme Ayala Marikina,Ayala Malls Marikina,For Opening,2024-04-15,MS1XI4A,NCR East,Albee David
16,Rulls,MBC Gaisano Inline Binangonan,Gaisano Inline Binangonan,For Opening,2024-04-15,M38S4QA,NCR East,Albee David
17,Rulls,Realme Robinsons Metro East,Robinsons Metro East,For Opening,2024-04-15,MBLR71F,NCR East,Albee David
19,Rulls,Mobile Craze PG Tanay Rizal,Puregold Tanay Rizal,For Opening,2024-04-15,MFCWRJQ,NCR East,Albee David


In [47]:
df.to_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\SIL Sales\Shin Nebria\POS_Addition\Data\POS_mapping_data.xlsx", index = False)

In [48]:
df[df['PurpleKey']=='MX1EHIO']

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
164,Rulls,Rulls Waltermart Bicutan,Standalone,For Opening,2024-07-26,MX1EHIO,NCR South 2,Grace Belarmino
